In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.applications.efficientnet import preprocess_input

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(400, 400, 3))
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

inputs = tf.keras.Input(shape=(400, 400, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(2, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer=AdamW(learning_rate=0.00005), loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.5,
    zoom_range=0.3,
    brightness_range=[0.5, 1.5],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow(x_train, y_train, batch_size=32)
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = valid_datagen.flow(x_test, y_test, batch_size=32)

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=30
)
